In [358]:
import pyedflib
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA


In [359]:
from load_data import Subject_Session_Data
#from preprocess import butter_bandpass_filter
#from feature_extraction import EEGFeatureExtraction

In [360]:
def load_dataset():
        work_dir = os.path.dirname(os.path.dirname(os.getcwd()))
        dataset_dir = os.path.join(work_dir, "enterface06_EMOBRAIN", "Data")
        dataset_common_dir = os.path.join(dataset_dir, "Common")
        dataset_EEG_dir = os.path.join(dataset_dir, "EEG")
        dataset_fNIRS_dir = os.path.join(dataset_dir, "fNIRS")

        print("working with dataset directory:", dataset_dir)

        # Construct the search pattern
        pattern = os.path.join(dataset_EEG_dir, "*.bdf")

        # Find all files in the directory matching the pattern
        bdf_files = glob.glob(pattern)

        count = 1
        data_list = []
        for file in bdf_files:
                # Open the BDF file
                try:
                        with pyedflib.EdfReader(file) as f:
                                # marks = []
                                print("reading file :", count)
                                participant_index = file.find("Part")
                                session_index = file.find("SES")
                                participant_number = int(file[participant_index+4])
                                # print("Participant number:", participant_number)
                                session_number = int(file[session_index+3])
                                # print("Session number:", session_number)
                                if participant_number == 2 and session_number == 1:
                                        continue
                                fi = open(file+".mrk", "r")
                                fi.readline()
                                inferred_temp_marks = []
                                for line in fi:
                                        temp_marks = line.split('\t')[1:]
                                        temp_marks[-1] = temp_marks[-1][:-1]
                                        if temp_marks[-1] == '"255"':
                                                if temp_marks[0] == temp_marks[1]:
                                                        inferred_temp_marks.append(int(temp_marks[0]))
                                                else:
                                                        print("Irregular:", temp_marks)

                                block_sample_ranges = []
                                for trigger in inferred_temp_marks:
                                        if participant_number == 1 and session_number == 1:
                                                start_index = trigger + 768
                                                end_index = start_index + 3200
                                                block_sample_ranges.append((start_index,end_index))
                                        else:
                                                start_index = trigger + 3072
                                                end_index = start_index + 12800
                                                block_sample_ranges.append((start_index,end_index))

                                data_list.append(Subject_Session_Data(f, count, participant_number, session_number, inferred_temp_marks, block_sample_ranges))
                                '''
                                # Get general information
                                print("reading file :", count)
                                print("File duration in seconds:", f.file_duration)
                                print("Number of signals:", f.signals_in_file)
                                signal_labels = f.getSignalLabels()
                                print("Signal labels:", signal_labels)

                                # Read data from each signal
                                for i in range(f.signals_in_file):
                                        data = f.readSignal(i)
                                        print(f"Data from signal {signal_labels[i]}:", data)
                                '''
                except (OSError):
                        print("reading file :", count, "FAIL")
                count += 1

        emotion_classes = os.path.join(dataset_common_dir, "IAPS_Classes_EEG_fNIRS.txt")
        emotion_file = open(emotion_classes, "r")
        session1 = []
        session2 = []
        session3 = []
        for line in emotion_file:
                temp_emotions = line.split("\t")
                temp_emotions[-1] = temp_emotions[-1][:-1]
                session1.append(temp_emotions[0])
                session2.append(temp_emotions[1])
                session3.append(temp_emotions[2])

        for subject_session in data_list:
                if subject_session.session_number == 1:
                        subject_session.emotions = session1
                elif subject_session.session_number == 2:
                        subject_session.emotions = session2
                elif subject_session.session_number == 3:
                        subject_session.emotions = session3

        ################################ VALIDATION OF .MRK FILE DO NOT DELETE ##############################
        # seconds = {}
        # j_index = 0
        # for j in range(len(data_list)):
        #         flag = 0
        #         print("Number of marks in session:", len(data_list[j].marks))
        #         print("Number of emotions:", len(data_list[j].emotions))
        #         print(data_list[j].marks)
        #         print("number of signals:", len(data_list[j].data_dictionary[data_list[j].signal_labels[0]]))
        #         print("participant number:", data_list[j].participant_number)
        #         print("session number:", data_list[j].session_number)
                
        #         for i in range(len(data_list[j].marks)):
        #                 if i == 0:
        #                         continue
        #                 else:
        #                         if data_list[j].participant_number == 1 and data_list[j].session_number == 1:
        #                                 temp_seconds = (data_list[j].marks[i] - data_list[j].marks[i-1])/256
        #                                 flag = 1
        #                         else:
        #                                 temp_seconds = (data_list[j].marks[i] - data_list[j].marks[i-1])/1024
        #                         if flag == 1:
        #                                 j_index = j

        #                         if j in seconds.keys():
        #                                 seconds[j].append(temp_seconds)
        #                         else:
        #                                 seconds[j] = [temp_seconds]

        # for key in seconds.keys():
        #         if key == j_index:
        #                 print("First participant, first session")
        #                 print("Minimum time for one trial:", min(seconds[key]))
        #                 print("Maximum time for one trial:", max(seconds[key]))
        #                 print("Mean time for one trial:", sum(seconds[key])/len(seconds[key]))
        #         else:
        #                 print("Minimum time for one trial:", min(seconds[key]))
        #                 print("Maximum time for one trial:", max(seconds[key]))
        #                 print("Mean time for one trial:", sum(seconds[key])/len(seconds[key]))

        return data_list

In [362]:
dataset = load_dataset()

working with dataset directory: c:\Users\LyuxingHe\Desktop\GitRepos\M2S\enterface06_EMOBRAIN\Data
reading file : 1
reading file : 2
reading file : 3
reading file : 4
reading file : 4
reading file : 5
reading file : 6
reading file : 7
reading file : 8
reading file : 9
reading file : 10
reading file : 11
reading file : 12
reading file : 13
reading file : 14


In [363]:
dataset[2].display_info()
sample_data = dataset[2]

file number: 3
File duration in seconds: 1158
Number of channels: 72
Number of samples: 1185792
Participant number: 1
Session number: 3
Emotions: ['Pos', 'Neg', 'Calm', 'Pos', 'Pos', 'Neg', 'Calm', 'Neg', 'Calm', 'Calm', 'Neg', 'Neg', 'Calm', 'Pos', 'Pos', 'Calm', 'Neg', 'Pos', 'Neg', 'Calm', 'Pos', 'Calm', 'Calm', 'Neg', 'Pos', 'Calm', 'Pos', 'Pos', 'Neg', 'Neg']
Signal labels: ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp', 'Status']


In [364]:
print(len(dataset))

14


In [365]:
# shows the 12.5s truncation index
print(sample_data.block_sample_ranges)

[(15970, 28770), (87669, 100469), (123319, 136119), (161059, 173859), (196729, 209529), (235734, 248534), (269850, 282650), (307086, 319886), (354439, 367239), (393163, 405963), (430615, 443415), (466212, 479012), (503643, 516443), (540469, 553269), (578306, 591106), (614546, 627346), (651215, 664015), (690152, 702952), (724799, 737599), (761431, 774231), (799082, 811882), (835172, 847972), (870946, 883746), (910581, 923381), (944273, 957073), (980509, 993309), (1014399, 1027199), (1049390, 1062190), (1082367, 1095167), (1118888, 1131688)]


In [375]:
import math
from scipy import signal

from scipy.signal import butter, sosfiltfilt

# Bandpass filter
# For: Environmental noise removal by applying a bandpass filter in the 4-45 Hz range
# fs: sample rate, lowcut/ highcut: band cutoffs
def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    #sos = butter(order, [low, high], analog=False, fs=fs, btype='band', output='sos')
    sos = butter(order, [lowcut, highcut], analog=False, fs=fs, btype='band', output='sos')
    filtered = sosfiltfilt(sos, data)
    return filtered

class EEGFeatureExtraction:
    
    def __init__(self, data, fs=1024):
        self.data = data
        self.fs = fs
        self.num_samples = len(data)
        self.num_channels = data[1].num_channels
        self.num_truncations = len(data[1].block_sample_ranges)
        self.target_channels = ['F3', 'F4', 'T7', 'Fp1', 'Fp2', 'T8', 'F7', 'F8', 'O1', 'P7', 'P8', 'O2']
        print()

    def display_info(self):
            # samples = {}
            print("Number of samples:", self.num_samples)
            print("Number of channels:", self.num_channels)
            print("Number of truncations:", self.num_truncations)
            print("Target channels:", self.target_channels)

    def extract_features(self):
        results = np.zeros(shape = (self.num_samples, len(self.target_channels) * self.num_truncations))
        for sample_id, sample in enumerate(self.data):
            for channel_id, channel in enumerate(self.target_channels):
                assert channel in sample.signal_labels, channel + " target channel is not present in the sample data!"
                channel_signal = sample.data_dictionary[channel]
                for band_id, truncation_range in enumerate(sample.block_sample_ranges):
                    truncation = channel_signal[truncation_range[0]:truncation_range[1]]
                    filtered = butter_bandpass_filter(truncation, 4, 45, self.fs)
                    f, Pxx_den = signal.welch(truncation, self.fs, nperseg=256, noverlap=128)
                    #print Pxx_den.shape, Pxx_den
                
                    results[sample_id][channel_id*self.num_truncations + band_id] = math.log(np.max(Pxx_den))

        return results

In [376]:
eeg_fe = EEGFeatureExtraction(dataset)
eeg_fe.display_info()


Number of samples: 14
Number of channels: 72
Number of truncations: 30
Target channels: ['F3', 'F4', 'T7', 'Fp1', 'Fp2', 'T8', 'F7', 'F8', 'O1', 'P7', 'P8', 'O2']


In [377]:
eeg_features = eeg_fe.extract_features()

In [378]:
eeg_labels = np.array([sample.emotions for sample in dataset])

In [379]:
print("feature shape:", eeg_features.shape)
print("label shape:", eeg_labels.shape)

feature shape: (14, 360)
label shape: (14, 30)


In [380]:
import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

reshaped_features = eeg_features.reshape(14*30, -1)  # (420, 12)
flattened_labels = eeg_labels.flatten()
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(flattened_labels)
integer_encoded = integer_encoded.reshape(-1, 1)

features_tensor = torch.tensor(reshaped_features, dtype=torch.float32)
labels_tensor = torch.tensor(integer_encoded, dtype=torch.long).squeeze()
print("feature shape:", features_tensor.shape)
print("label shape:", labels_tensor.shape)
print("Unique encoded labels:", np.unique(integer_encoded))

tensor_dataset = TensorDataset(features_tensor, labels_tensor)

train_dataset, test_dataset = train_test_split(tensor_dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

feature shape: torch.Size([420, 12])
label shape: torch.Size([420])
Unique encoded labels: [0 1 2]


In [421]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(12, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.fc5 = nn.Linear(128, 3)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn3(self.fc4(x)))
        x = self.dropout(x)

        x = self.fc5(x)
        return x

model = SimpleNN()

In [422]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5000):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

Epoch 1, Loss: 1.1842227122362923
Epoch 2, Loss: 1.1474052772802465
Epoch 3, Loss: 1.1768333754118752
Epoch 4, Loss: 1.1474674361593582
Epoch 5, Loss: 1.171073028269936
Epoch 6, Loss: 1.1927397145944483
Epoch 7, Loss: 1.129572051412919
Epoch 8, Loss: 1.136613344444948
Epoch 9, Loss: 1.1455895918257095
Epoch 10, Loss: 1.1201479750521042
Epoch 11, Loss: 1.1538224570891435
Epoch 12, Loss: 1.118999390041127
Epoch 13, Loss: 1.1144402272561018
Epoch 14, Loss: 1.128055197351119
Epoch 15, Loss: 1.1260192885118372
Epoch 16, Loss: 1.127236832590664
Epoch 17, Loss: 1.1174578579033123
Epoch 18, Loss: 1.1267189611406887
Epoch 19, Loss: 1.1135926421950846
Epoch 20, Loss: 1.106317763819414
Epoch 21, Loss: 1.1071550477953518
Epoch 22, Loss: 1.106921259094687
Epoch 23, Loss: 1.1352115624091204
Epoch 24, Loss: 1.0998321929398704
Epoch 25, Loss: 1.1048171029371374
Epoch 26, Loss: 1.087367850191453
Epoch 27, Loss: 1.1021498897496391
Epoch 28, Loss: 1.1135886679677403
Epoch 29, Loss: 1.1198573007303125
Epo

In [428]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test data: {100 * correct // total}%')

Accuracy of the network on the test data: 34%


In [ ]:
#######################################################################
#######################################################################